In [1]:
from PIL import Image
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import RYGate, QFT
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session
import numpy as np
from qiskit import transpile
# Configuration flags (user can toggle these)
ENCODING_MODE = 'phase'  # 'amplitude', 'phase', or 'combined'
FULL_SOBEL = True           # True for full 8-direction Sobel, False for simpler Sobel
#Api_token= "8517bec216407d31487d5cb2ce893b43f356a831180bb0807a94effc8f4244187a435a2204a1a78574ac63fb7bea585e87db1165468f943218e9ddbcab3143b3" 
#Api_token = "e91e4f3b226dbbd64380f23b57b9a3a614a8274acfafa90cd3eb4d22157e57431e4d0e401b220baa9bf6178388221617d046ae72723756381e32c41a7b118271"
#Api_token = "4c44168dc11baf072bae7008bdae971711605d63c82c5fd010aee95d71fe8b55e05e41318e80dc353ad0eace7c96090b5b0c83fc1c2f0bb97106a0d56bb5b6d0"
#Api_token = "4db56cee63880ff5a5400c86f26e9ea2af78e3af7b1fdcd1b8d1fd3e738336234c42aa81b6758de8bfc76dc26ce4c91b5a1c9e3c5bd66cc189761282fb983512"
Api_token = "f89c3328c0bbb4c105f4c264d77759a75df42474fecaf95f5011ed54f3a956993a11c23eaa18750a3c06cc4a103766813eb32dab133e734d9ca04778afd4f029"
def encode_frqi_block(image_block, encoding='amplitude'):
    """
    Encode a single grayscale image block into an FRQI quantum state.
    - image_block: 2D numpy array of grayscale intensities (0-255).
    - encoding: 'amplitude', 'phase', or 'combined'.
    Returns: QuantumCircuit with FRQI encoding prepared (with a color qubit and position qubits).
    """
    flat = image_block.flatten().astype(float)
    
    if flat.size <= 1:
        raise ValueError("Block too small for FRQI encoding.")
    
    # Normalize intensities to [0,1]
    flat = flat / float(np.max(flat)) if np.max(flat) > 0 else flat
    num_pixels = flat.size
    # Determine number of position qubits (must cover all pixels)
    num_pos_qubits = int(np.ceil(np.log2(num_pixels)))
    # Create quantum registers: one 'color' qubit and num_pos_qubits for position
    color = QuantumRegister(1, name='color')
    pos = QuantumRegister(num_pos_qubits, name='pos')
    # Classical registers for measurements
    color_c = ClassicalRegister(1, name='c_color')
    pos_c = ClassicalRegister(num_pos_qubits, name='c_pos')
    qc = QuantumCircuit(color, pos, color_c, pos_c)
    # Initialize position qubits to equal superposition (Hadamard on each)
    qc.h(pos)
    # Encode intensities: for each pixel, apply controlled rotation on color qubit
    for idx, intensity in enumerate(flat):
        theta = np.arcsin(intensity)  # angle encoding for FRQI
        bin_str = format(idx, '0{}b'.format(num_pos_qubits))
        # Flip pos qubits to create control for this index
        for qb, bit in zip(pos, bin_str):
            if bit == '0':
                qc.x(qb)
        # Apply the controlled rotation on color qubit
        if encoding == 'amplitude':
            qc.mcry(2 * theta, pos[:], color[0], None)
        elif encoding == 'phase':
            qc.mcrz(2 * theta, pos[:], color[0], None)
        elif encoding == 'combined':
            qc.mcry(2 * theta, pos[:], color[0], None)
            qc.mcrz(2 * theta, pos[:], color[0], None)
        else:
            raise ValueError("Unsupported encoding mode: choose 'amplitude', 'phase', or 'combined'.")
        # Undo the flips
        for qb, bit in zip(pos, bin_str):
            if bit == '0':
                qc.x(qb)
    return qc

def quantum_sobel_filter(qc, full=True):
    """
    Append a Quantum Sobel filter to the FRQI circuit qc.
    - qc: QuantumCircuit with FRQI-encoded block (color and pos registers).
    - full: True to apply full 8-direction QFT-based Sobel, False for a simplified version.
    (Modifies qc in place.)
    """
    # Identify qubits: assume first qubit is color, the rest are position
    color_qubit = qc.qubits[0]
    pos_qubits = qc.qubits[1:]
    if full:
        # Apply Quantum Fourier Transform on position qubits
        qft = QFT(len(pos_qubits), do_swaps=False, inverse=False)
        qc.append(qft.to_instruction(), pos_qubits)
        # Apply phase rotations as a stand-in for convolution in frequency domain
        for i, qb in enumerate(pos_qubits):
            qc.p(2 * np.pi * (i + 1) / (2**len(pos_qubits)), qb)
        # Inverse QFT to go back to spatial domain
        qc.append(qft.inverse().to_instruction(), pos_qubits)
    else:
        # Simplified Sobel: e.g., mark edges with an X (placeholder for simpler gradient)
        qc.x(color_qubit)

def split_image_into_blocks(image, block_size):
    """
    Split image into non-overlapping blocks of given size.
    Returns lists of blocks and their top-left coordinates.
    """
    blocks, coords = [], []
    h, w = image.shape
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = image[i:i+block_size, j:j+block_size]
            blocks.append(block)
            coords.append((i, j))
    return blocks, coords

def recombine_blocks(edge_blocks, coords, image_shape):
    """
    Reassemble blocks into a full edge-detected image.
    - edge_blocks: list of 2D numpy arrays (edge intensities for each block).
    - coords: list of (row, col) coordinates for each block.
    - image_shape: tuple (height, width) of original image.
    """
    edge_image = np.zeros(image_shape, dtype=float)
    for block, (i, j) in zip(edge_blocks, coords):
        h, w = block.shape
        edge_image[i:i+h, j:j+w] = block
    return edge_image



In [2]:

#image = np.array(Image.open('/workspaces/AQA/images/Quantum-edge-test-photo.png').convert('L'))
#image = np.array(Image.open('/workspaces/AQA/images/Edge-Test-2.jpg').convert('L'))
image= np.array(Image.open('/workspaces/AQA/images/Edge-Test-3.png').convert('L'))
# Split image into large blocks
large_block_size = 256
small_block_size = 128
shots = 128

large_blocks, large_coords = split_image_into_blocks(image, large_block_size)
all_small_blocks = []
all_small_coords = []
large_shapes = []

for large_block in large_blocks:
    small_blocks, small_coords = split_image_into_blocks(large_block, small_block_size)
    all_small_blocks.extend(small_blocks)
    all_small_coords.extend(small_coords)
    large_shapes.append(large_block.shape)

In [3]:
def batch(iterable, n=10):
    """Yield successive n-sized batches from iterable."""
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

In [4]:
def crop_or_pad_to_256(image):
    """Crop or pad a 2D numpy array to 256x256 (centered)."""
    h, w = image.shape
    # Crop if needed
    if h > 256:
        top = (h - 256) // 2
        image = image[top:top+256, :]
    if w > 256:
        left = (w - 256) // 2
        image = image[:, left:left+256]
    # Pad if needed
    h, w = image.shape
    pad_h = max(0, 256 - h)
    pad_w = max(0, 256 - w)
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left
    return np.pad(image, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')

# Usage:
image = crop_or_pad_to_256(image)

print("Padded image shape:", image.shape)

Padded image shape: (256, 256)


In [ ]:
import math

print(f"Image shape: {image.shape}")
print(f"Large block size: {large_block_size}")
print(f"Small block size: {small_block_size}")

# Calculate block counts
num_large_blocks = (image.shape[0] // large_block_size) * (image.shape[1] // large_block_size)
num_small_blocks_per_large = (large_block_size // small_block_size) ** 2
total_small_blocks = num_large_blocks * num_small_blocks_per_large

print(f"Total large blocks: {num_large_blocks}")
print(f"Small blocks per large block: {num_small_blocks_per_large}")
print(f"Total small blocks (circuits): {total_small_blocks}")

batch_size = 8  # or whatever you set
print(f"Batch size: {batch_size}")
print(f"Expected QPU jobs: {math.ceil(total_small_blocks / batch_size)}")

Image shape: (256, 256)
Large block size: 256
Small block size: 128
Total large blocks: 1
Small blocks per large block: 4
Total small blocks (circuits): 4
Batch size: 8
Expected QPU jobs: 1


: 

In [ ]:
# Prepare circuits for all small blocks
circuits = []
for small_block in all_small_blocks:
    if small_block.size <= 1:
        continue
    qc = encode_frqi_block(small_block, encoding=ENCODING_MODE)
    quantum_sobel_filter(qc, full=FULL_SOBEL)
    qc.measure(qc.qubits, qc.clbits)
    circuits.append(qc)
# QPU execution (batch or in chunks if needed)


In [ ]:
import math

print(f"Image shape: {image.shape}")
print(f"Large block size: {large_block_size}")
print(f"Small block size: {small_block_size}")
print(f"Total large blocks: {len(large_blocks)}")
print(f"Total small blocks: {len(all_small_blocks)}")
print(f"Total circuits: {len(circuits)}")
print(f"Batch size: {batch_size}")
print(f"Expected QPU jobs: {math.ceil(len(circuits) / batch_size)}")

In [ ]:
    
QiskitRuntimeService.save_account(channel="ibm_quantum", token=Api_token, overwrite=True)
service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")


In [ ]:
for circ_batch in batch(circuits, batch_size):
    circuits_transpiled = transpile(circ_batch, backend=backend, optimization_level=1)
    with Session(backend=backend) as session:
        sampler = Sampler()
        job = sampler.run(circuits_transpiled, shots=shots)
        result = job.result()
        all_pub_results.extend(result._pub_results) 

In [ ]:
from collections import Counter

all_counts = []
for pub_result in all_pub_results:
    data = pub_result.data
    color_bits = data.c_color.array  # shape: (shots, 1)
    pos_bits = data.c_pos.array      # shape: (shots, num_pos_bits)
    # Robust conversion for both 1D and 2D pos_bits
    if len(pos_bits.shape) == 2:
        pos_ints = [int("".join(str(int(b)) for b in bits), 2) for bits in pos_bits]
    else:
        pos_ints = [int(b) for b in pos_bits]
    color_ints = [int(b[0]) for b in color_bits]
    bitstrings = [str(color) + format(pos, f'0{pos_bits.shape[-1]}b') for color, pos in zip(color_ints, pos_ints)]
    counts = Counter(bitstrings)
    all_counts.append(counts)

In [ ]:
sub_edge_blocks = []
for counts in all_counts:
    total_shots = sum(counts.values())
    block_edges = np.zeros((small_block_size, small_block_size))
    for bitstring, count in counts.items():
        color_bit = int(bitstring[0])
        pos_bits = bitstring[1:]
        pos_index = int(pos_bits, 2) if pos_bits else 0
        row = pos_index // small_block_size
        col = pos_index % small_block_size
        if color_bit == 1:
            block_edges[row, col] += count / total_shots
    sub_edge_blocks.append(block_edges)

ValueError: invalid literal for int() with base 2: '2'

In [ ]:
# Process results for each small block
sub_edge_blocks = []
for idx, quasi_dist in enumerate(results.quasi_dists):
    counts = {k: int(v * shots) for k, v in quasi_dist.items()}
    total_shots = sum(counts.values())
    block_edges = np.zeros((small_block_size, small_block_size))
    for bitstring, count in counts.items():
        color_bit = int(bitstring[0])
        pos_bits = bitstring[1:]
        pos_index = int(pos_bits, 2) if pos_bits else 0
        row = pos_index // small_block_size
        col = pos_index % small_block_size
        if color_bit == 1:
            block_edges[row, col] += count / total_shots
    sub_edge_blocks.append(block_edges)

# Now reassemble small blocks into large blocks, then into the full image
edge_blocks = []
idx = 0
for shape in large_shapes:
    n_sub_blocks = (shape[0] // small_block_size) * (shape[1] // small_block_size)
    sub_blocks = sub_edge_blocks[idx:idx + n_sub_blocks]
    coords = []
    for i in range(0, shape[0], small_block_size):
        for j in range(0, shape[1], small_block_size):
            coords.append((i, j))
    large_block_edge = recombine_blocks(sub_blocks, coords, shape)
    edge_blocks.append(large_block_edge)
    idx += n_sub_blocks

# Recombine all large blocks into the final image
edge_image = recombine_blocks(edge_blocks, large_coords, image.shape)
print("Edge image shape:", edge_image.shape)

In [ ]:


print("Edge image shape:", edge_image.shape)
block_size = 32 # ensure power-of-2 dimensions for blocks
blocks, coords = split_image_into_blocks(image, block_size)
circuits = []
# Build a quantum circuit for each block
for block in blocks:
    qc = encode_frqi_block(block, encoding=ENCODING_MODE)
    quantum_sobel_filter(qc, full=FULL_SOBEL)
    # Measure all qubits (color + position) into classical bits
    qc.measure(qc.qubits, qc.clbits)
    circuits.append(qc)
# Execute circuits on IBM hardware (or simulator) with Qiskit Runtime Sampler



: 